In [ ]:
!pip install TensorFlow
!pip install Keras
!pip install pillow
!pip install NumPy
!pip install tqdm
!pip install jupyterlab
!pip install pydot

In [ ]:
import numpy as np
from PIL import Image
import os, fnmatch
import string, random
from pickle import dump
from pickle import load
from keras.applications.xception import Xception 
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.layers import LSTM, Embedding, Dropout
from tqdm.notebook import tqdm as tqdm 
from nltk.translate.bleu_score import corpus_bleu

tqdm().pandas()

0it [00:00, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import glob

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/w266"

Archive.zip	  features_Xc.p		      model_results_64_epochs.csv
descriptions.txt  Flicker8kDataset	      Xcp_tokenizer.p
features.p	  Flicker8kText
features_vgg.p	  model_results_30_epocs.csv


In [ ]:
image_path = "/content/drive/My Drive/w266/Flicker8kDataset"
files = glob.glob(image_path+ "/*.jpg")
i=0
for file in files:
    i=i+1

print("Total Images in Dataset", i)

Total Images in Dataset 8091


In [ ]:
# Read the contents of the file into a string.
def load_fp(filename):
    print("load_fp:",filename)

    with open(filename, 'r') as f:
        text = f.read()
        print("size of file:",f.tell())
        f.close()
    return text

In [ ]:
# Description dictionary that will map images with all 5 captions
import re
def img_capt(filename):
    file = load_fp(filename)
    captions = file.split('\n')
    descriptions ={}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [caption]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions

In [ ]:
'''Data cleaning function will convert all upper case alphabets to lowercase, removing punctuations and words containing numbers
This method is used to clean the data by taking all descriptions as input. 
While dealing with textual data we need to perform several types of cleaning 
including uppercase to lowercase conversion, punctuation removal, and removal 
of the number containing words.
'''
def txt_clean(captions):
    table = str.maketrans('','',string.punctuation)
    for img,caps in captions.items():
        for i,img_caption in enumerate(caps):
            img_caption.replace("-"," ")
            descp = img_caption.split()
          #uppercase to lowercase
            descp = [wrd.lower() for wrd in descp]
          #remove punctuation from each token
            descp = [wrd.translate(table) for wrd in descp]
          #remove hanging 's and a
            descp = [wrd for wrd in descp if(len(wrd)>1)]
          #remove words containing numbers with them
            descp = [wrd for wrd in descp if(wrd.isalpha())]
          #converting back to string
            img_caption = ' '.join(descp)
            captions[img][i]= img_caption
    return captions


In [ ]:
'''This function is used to create a vocabulary from all the unique words extracted out from descriptions.
'''
def txt_vocab(descriptions):
    vocab = set()
    for key in descriptions.keys():
       [vocab.update(d.split()) for d in descriptions[key]]
    return vocab

In [ ]:
#To save all descriptions in one file
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc )
    data = "\n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()

In [ ]:
# set folder locations to source images and desctions
dataset_text = "/content/drive/My Drive/w266/Flicker8kText"
dataset_images = "/content/drive/My Drive/w266/Flicker8kDataset"
dataset_base = "/content/drive/My Drive/w266"

In [ ]:

filename = dataset_text + "/" + "Flickr8k.token.txt"

#build descriptions dictionary
descriptions = img_capt(filename)
print("Length of descriptions =" ,len(descriptions))


#cleaning the descriptions
clean_descriptions = txt_clean(descriptions)
print("Length of clean descriptions =" ,len(descriptions))

#Build vocabulary
vocabulary = txt_vocab(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))

#Save descriptions for future use
save_descriptions(clean_descriptions, dataset_base+"/descriptions.txt")
print("Descriptions saved as descriptions.txt")


In [ ]:
import os, fnmatch
files = fnmatch.filter(os.listdir(dataset_images), '*.jpg')


8091


In [ ]:
def extract_features(directory):
        model = Xception( include_top=False, pooling='avg' )
        features = {}
        files = fnmatch.filter(os.listdir(directory), '*.jpg')
        for img in tqdm(files):
            filename = directory + "/" + img
            image = Image.open(filename)
            X.append(img)
            image = image.resize((299,299))
            image = np.expand_dims(image, axis=0)
            image = image/127.5
            image = image - 1.0
            
            feature = model.predict(image)
            features[img] = feature
        return features

In [ ]:
X=[]
features = extract_features(dataset_images)

  0%|          | 0/8091 [00:00<?, ?it/s]

In [ ]:
# Data preparation with shuffle
random.shuffle(X)
train_size = int(len(files)* .8)
train_arr = X[:train_size]
test_arr = X[train_size:]


In [ ]:
dump(features, open(dataset_base+"/features_Xc.p","wb"))


In [ ]:
features = []
features = load(open(dataset_base+"/features_Xc.p","rb"))

In [ ]:
output_file = open(dataset_text + "/" + "Flickr_8k.trainImages_Xc.txt", 'w')

for fname in train_arr:
    output_file.write(fname + '\n')

output_file.close()

output_file = open(dataset_text + "/" + "Flickr_8k.testImages_Xc.txt", 'w')

for fname in test_arr:
    output_file.write(fname + '\n')

output_file.close()

In [ ]:
'''This function will take a file name as a parameter and return the list of image names by 
loading the text file into a string.
'''
def load_photos(filename):
    file = load_fp(filename)
    photos = file.split("\n")[:-1]
    return photos

In [ ]:
'''Storing the captions for every image from the list of photos to a dictionary. 
For the ease of the LSTM model in identifying the beginning and ending of a caption, 
we appended the and identifier with each caption.
'''
def load_clean_descriptions(filename, photos):

    file = load_fp(filename)
    descriptions = {}
    for line in file.split("\n"):
        words = line.split()
        if len(words)<1 :
            continue
            
        image, image_caption = words[0], words[1:]
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
                desc = '<start> ' + " ".join(image_caption) + ' <end>'
                descriptions[image].append(desc)
    return descriptions

In [ ]:
'''The extracted feature vectors from the Xception model and the dictionary for 
photos are returned by this function.
'''
def load_features(photos):
    all_features = load(open(dataset_base+"/features.p","rb"))
    features = {k:all_features[k] for k in photos}
    all_features = []
    return features


In [ ]:
filename = dataset_text + "/" + "Flickr_8k.trainImages_Xc.txt"
train_imgs = load_photos(filename)
print("size of train_imgs=", len(train_imgs))

train_descriptions = load_clean_descriptions(dataset_base+"/descriptions.txt", train_imgs)
print("size of train_descriptions =", len(train_descriptions))

train_features = load_features(train_imgs)
print("size of train_features =", len(train_features))


In [ ]:
''' convert dictionary to list of descriptions '''
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

In [ ]:
from keras.preprocessing.text import Tokenizer
def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer

In [ ]:
''' Give each word an index, and store that into tokenizer.p pickle file. This is to make sure 
we dont have to build the model everytime we start the process'''
tokenizer = create_tokenizer(train_descriptions)
dump(tokenizer, open(dataset_base + '/Xcp_tokenizer.p', 'wb'))
vocab_size = len(tokenizer.word_index) + 1
vocab_size 

4020

In [ ]:
''' calculate maximum length of descriptions to decide the model structure parameters. '''
def get_max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)

max_length = get_max_length(descriptions)

In [ ]:
''' Generic method to get data for model '''
def data_generator(descriptions, features, tokenizer, max_length):
    while 1:
        for key, description_list in descriptions.items():
            feature = features[key][0]
            inp_image, inp_seq, op_word = create_sequences(tokenizer, max_length, description_list, feature)
            yield [[inp_image, inp_seq], op_word]

In [ ]:
def create_sequences(tokenizer, max_length, desc_list, feature):
    x_1, x_2, y = list(), list(), list()
    for desc in desc_list:
        seq = tokenizer.texts_to_sequences([desc])[0]
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            x_1.append(feature)
            x_2.append(in_seq)
            y.append(out_seq)
    return np.array(x_1), np.array(x_2), np.array(y)

In [ ]:

[a,b],c = next(data_generator(train_descriptions, features, tokenizer, max_length))
a.shape, b.shape, c.shape

((8, 2048), (8, 32), (8, 4020))

In [ ]:
from tensorflow.keras.utils import plot_model
# Model definition
def define_model(vocab_size, max_length):
  # features from the CNN model compressed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
  # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
  # Merging both models
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
  # merge it [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    plot_model(model, to_file=dataset_base + '/model.png', show_shapes=True)
    return model

Various model validation and evaluation steps. Feature extraction, description generation are used for the final test

In [ ]:
def extract_features(filename, model):
    try:
         image = Image.open(filename)
    except:
        print("ERROR: Can't open image! Ensure that image path and extension is correct")
 
    image = image.resize((299,299))
    image = np.array(image)
    if image.shape[2] == 4:
        image = image[..., :3]
    image = np.expand_dims(image, axis=0)
    image = image/127.5
    image = image - 1.0
    feature = model.predict(image)
    return feature

In [ ]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [ ]:
def generate_description(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

In [ ]:
''' Exception Model Training '''
model = []
model = define_model(vocab_size, max_length)
epochs = 10
steps = len(train_descriptions)
print("steps=", steps)
# Saving models to a dictionary
try:
    os.rmdir(dataset_base + "/modelsxcp")
    print("Removed Directory models..")
except:
    print("Directory does not exist..")

print("Creating directory..")
os.mkdir(dataset_base + "/modelsxcp")
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch= steps, verbose=1)
    model.save(dataset_base + "/modelsxcp/model_" + str(i) + ".h5")

In [ ]:
''' Model evaluation '''
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    # step over the whole set
    for key, desc_list in descriptions.items():
        # generate description
        yhat = generate_description(model, tokenizer, photos[key], max_length)
        # store actual and predicted
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat.split())
    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))



In [ ]:
filename = dataset_text + "/" + "Flickr_8k.testImages_Xc.txt"
test_imgs = load_photos(filename)
print("size of test_imgs=", len(test_imgs))

test_descriptions = load_clean_descriptions(dataset_base+"/descriptions.txt", test_imgs)
print("size of test_descriptions =", len(test_descriptions))

test_features = load_features(test_imgs)
print("size of test_features =", len(test_features))


In [ ]:
model = load_model(dataset_base + '/modelsxcp/model_9.h5')
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

In [ ]:
import matplotlib.pyplot as plt
model = load_model(dataset_base + '/modelsxcp/model_9.h5')
fig = plt.figure(figsize=(10,20))
npic = 10
count = 1
xception_model = Xception(include_top=False, pooling="avg")

for sample_pic in test_imgs[20:30]:
    filename = dataset_images+"/"+sample_pic
    photo = extract_features(filename, xception_model)
    img = Image.open(filename)
    sample_description = generate_description(model, tokenizer, photo, max_length)
    ax = fig.add_subplot(npic,2,count,xticks=[],yticks=[])
    ax.imshow(img)
    count += 1
    sample_description = sample_description.lstrip('start ')
    sample_description = sample_description.rstrip(' end')
    actual_description = test_descriptions.get(sample_pic)[0]
    actual_description = actual_description.lstrip('<start> ')
    actual_description = actual_description.rstrip(' <end>')
    
    desc = "Actual caption: " + actual_description + "\n" + "Predicted caption: " + sample_description
    
    ax = fig.add_subplot(npic,2,count)
    plt.axis('off')
    ax.plot()
    ax.set_xlim(0,1)
    ax.set_ylim(0,len(desc))
    ax.text(0,count,desc,fontsize=20)
    
    count += 1
    print("\n\n")
plt.show()

In [ ]:

from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

In [ ]:
''' extract features from each photo in the directory '''
def extract_all_features_vgg(dirc):
    print("extract_all_features_vgg dirc=",dirc)
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    features = dict()
    files = fnmatch.filter(os.listdir(dirc), '*.jpg')
    for pic in tqdm(files):
        filename = dirc + '/' + pic
        image = Image.open(file)
        image = image.resize((224,224))
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
        features[pic] = feature
    return features


In [ ]:
features = extract_all_features_vgg(dataset_images)

In [ ]:
dump(features, open(dataset_base+"/features_vgg.p","wb"))
features = []
features = load(open(dataset_base+"/features_vgg.p","rb"))

In [ ]:
train_features = load_features(train_imgs)
print("size of train_features =", len(train_features))

test_features = load_features(test_imgs)
print("size of test_features =", len(test_features))

size of train_features = 6472
size of test_features = 1619


In [ ]:

[a,b],c = next(data_generator(train_descriptions, features, tokenizer, max_length))
a.shape, b.shape, c.shape

In [ ]:
''' VGG Model Training '''
model = []
model = define_model(vocab_size, max_length)
epochs = 10
steps = len(train_descriptions)
print("steps=", steps)
# creating a directory named models to save our models
try:
    os.rmdir(dataset_base + "/modelsvgg")
    print("Removed Directory models..")
except:
    print("Directory does not exist..")

print("Creating directory..")
os.mkdir(dataset_base + "/modelsvgg")
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch= steps, verbose=1)
    model.save(dataset_base + "/modelsvgg/model_" + str(i) + ".h5")

Various model validation and evaluation steps. Feature extraction, description generation are used for the final test. This set os for VGG

In [ ]:
def extract_vgg_features(filename, model):
    try:
         image = Image.open(filename)
    except:
        print("ERROR: Can't open image! Ensure that image path and extension is correct")
    
    image = image.resize((224,224))
    image = img_to_array(image)
    if image.shape[2] == 4:
        image = image[..., :3]

    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    return feature

In [ ]:
def evaluate_vgg_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    for key, desc_list in descriptions.items():
        yhat = generate_vgg_desc(model, tokenizer, photos[key], max_length)
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat.split())
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))


In [ ]:
def generate_vgg_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

In [ ]:
model = load_model(dataset_base + '/modelsvgg/model_9.h5')

In [ ]:
evaluate_vgg_model(model, test_descriptions, test_features, tokenizer, max_length)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,20))
npic = 10
count = 1
vgg_model = VGG16()

for sample_pic in test_imgs[20:30]:
    filename = dataset_images+"/"+sample_pic
    photo = extract_vgg_features(filename, vgg_model)
    img = Image.open(filename)
    pred_description = generate_vgg_desc(model, tokenizer, photo, max_length)
    ax = fig.add_subplot(npic,2,count,xticks=[],yticks=[])
    ax.imshow(img)
    count += 1
    pred_description = pred_description.lstrip('start ')
    pred_description = pred_description.rstrip(' end')
    actual_description = test_descriptions.get(sample_pic)[0]
    actual_description = actual_description.lstrip('<start> ')
    actual_description = actual_description.rstrip(' <end>')
    
    desc = "Actual caption: " + actual_description + "\n" + "Predicted caption: " + pred_description
    
    ax = fig.add_subplot(npic,2,count)
    plt.axis('off')
    ax.plot()
    ax.set_xlim(0,1)
    ax.set_ylim(0,len(desc))
    ax.text(0,count,desc,fontsize=20)
    
    count += 1
    print("\n\n")
#    print(sample_description)
plt.show()

In [ ]:
result_join = ' '.join("A B C D E F")
result_final = result_join.rsplit(' ', 1)[0]
print(result_join)
print(result_final.split())
print(result_final)
references = [d.split() for d in result_final]
print(references)

A   B   C   D   E   F
['A', 'B', 'C', 'D', 'E']
A   B   C   D   E  
[['A'], [], [], [], ['B'], [], [], [], ['C'], [], [], [], ['D'], [], [], [], ['E'], [], []]
